# BERT Embeddings
## Featuring: DistilBERT on PyTorch

This is adapted from HuggingFace example codes: https://huggingface.co/docs/transformers/model_doc/distilbert

### Preliminaries: Load the text tokenizer and deep learning model

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

### Create some input text and convert to tokens

In [ ]:
sentences = ["Hey, my dog is cute",
             "Hey, my dog is ugly",
             "Hey, my dog is not cute",
             "I put my money in the bank",
             "Banks have lots of money",
             "Hey, my dog is by river bank"]
inputs = tokenizer(sentences,
                   padding=True,
                   return_tensors="pt")
inputs

### Process the inputs and obtain the CLS token embeddings (whole sentence meaning)

Note that the text information for each sentence is now represented as a 768-dimensional vector. Similar vectors (close according to distance and/or dot-product computations) should be found for text with similar meanings. However, dissimilar vectors should correspond to dissimilar texts.

In [ ]:
with torch.no_grad():
    output = model(**inputs).last_hidden_state[:,0,:]
output.shape

### Quick, unscaled dot-product comparison between all embeddings

Note that higher values indicate more similarity when using dot-product. We could also calculate distances, which might be more appropriate in some cases.

In [ ]:
output @ output.T

### Euclidean distances between all embeddings

In [ ]:
from scipy.spatial.distance import pdist, squareform
squareform(pdist(output.numpy()))

In [ ]:
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
offset = 0.1
mds = MDS(n_components=2,dissimilarity='euclidean')
projection = mds.fit_transform(output.numpy())
plt.scatter(projection[:,0],projection[:,1])
for i,x in enumerate(projection):
    plt.text(x[0]+offset,x[1],'%d - %s'%(i,sentences[i]))
plt.axis('equal')
plt.show()

### Information distances between all embeddings

In [ ]:
logits = -1.0 * torch.log(((torch.softmax(output @ output.T,0) *
                            torch.softmax(output @ output.T,1))))
logits

In [ ]:
logits[:,0]

In [ ]:
logits[0,:]

In [ ]:
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
offset = 0.1
mds = MDS(n_components=2,dissimilarity='precomputed')
projection = mds.fit_transform(logits.numpy())
plt.scatter(projection[:,0],projection[:,1])
for i,x in enumerate(projection):
    plt.text(x[0]+offset,x[1],'%d - %s'%(i,sentences[i]))
plt.axis('equal')
plt.show()

### Obtain some summary/visualization info on the model...

In [ ]:
from torchinfo import summary
from torchview import draw_graph

In [ ]:
summary(model)

In [ ]:
model_graph = draw_graph(model,input_data=inputs)
model_graph.visual_graph